<a href="https://colab.research.google.com/github/dae1jeong/SSU_25_NLP_project/blob/main/notice%20crawling/notice_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import requests
from bs4 import BeautifulSoup
import json
import re

def crawl_post_content(url):
    """단일 게시글 URL에서 본문 텍스트, 이미지, 링크를 추출하여 반환"""

    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # 게시글 전체 영역 지정
        post_wrapper = soup.find('div', class_='bg-white p-4 mb-5')
        if not post_wrapper:
            return None

        # --- 메타데이터 추출 (옵션) ---
        # --- (생략: 메타데이터 추출 부분 - 이 함수에서는 반환하지 않고 목록 크롤러에서 사용) ---

        # --- 본문 영역 지정 (HR 태그 이후) ---
        # hr 태그를 찾고, 그 다음부터 본문이 시작되는 div를 찾습니다.
        hr_tag = post_wrapper.find('hr', class_='mb-4 mt-4')

        # hr_tag 바로 다음 형제(sibling) 요소를 본문 컨테이너로 간주
        content_container = hr_tag.find_next_sibling('div') if hr_tag else None

        if not content_container:
            print(f"경고: 본문 콘텐츠 영역을 찾을 수 없습니다. URL: {url}")
            return {'full_body_text': '본문 없음', 'image_urls': []}

        # --- 본문 내용 추출 및 데이터 수집 ---
        full_text_content = []
        image_urls = []

        # p, ol, ul, table 등 본문을 구성하는 모든 요소를 순회
        for element in content_container.find_all(recursive=False):

            # A. 테이블 처리
            if element.name == 'table':
                table_text = "\n[TABLE START] "
                headers = [th.get_text(strip=True) for th in element.find_all('th')]

                for row in element.find_all('tr'):
                    row_data = [td.get_text(strip=True).replace('\n', ' ') for td in row.find_all('td')]
                    if row_data:
                        # 헤더와 데이터 연결하여 텍스트로 변환
                        row_structure = " | ".join(f"{h}: {d}" for h, d in zip(headers, row_data))
                        table_text += row_structure + " / "

                full_text_content.append(table_text + " [TABLE END]\n")

            # B. 순서 목록 (<ol>, <ul>) 처리
            elif element.name in ['ol', 'ul']:
                list_text = "[LIST START] "
                for i, item in enumerate(element.find_all('li', recursive=False)):
                    list_text += f"({i+1}. {item.get_text(strip=True)}) "
                full_text_content.append(list_text + " [LIST END]\n")

            # C. 일반 단락/링크/이미지 (<p>, <a>, <img>) 처리
            elif element.name in ['p', 'div']:
                # 이미지 URL 추출
                for img_tag in element.find_all('img'):
                    img_src = img_tag.get('src')
                    if img_src and img_src not in image_urls:
                        image_urls.append(img_src)

                # 순수 텍스트 추출 (링크, 이미지 태그 제거 후)
                # element.get_text()는 자식 태그의 텍스트도 모두 가져오므로 여기서 추출
                text = element.get_text(strip=True)
                text = text.replace('\n', ' ').replace('\xa0', '').strip()
                if text:
                    # 텍스트 강조 처리 (선택 사항)
                    text = re.sub(r'(\w+)\s*:', r'**\1**:', text)
                    full_text_content.append(text)


        # 최종 텍스트 콘텐츠를 하나의 문자열로 결합
        final_text = "\n".join(full_text_content)
        final_text = re.sub(r'\n+', '\n', final_text).strip() # 연속된 줄바꿈 제거

        # 이 함수는 이 두 필드를 반환합니다.
        return {
            'full_body_text': final_text,
            'image_urls': image_urls
        }

    except requests.exceptions.RequestException as e:
        print(f"상세 크롤링 오류 발생: {url}, 오류: {e}")
        return None
    except Exception as e:
        print(f"상세 크롤링 중 알 수 없는 오류: {url}, 오류: {e}")
        return None

# # 예시 사용법
# # data = crawl_post_content("https://scatch.ssu.ac.kr/%EA%B3%B5%EC%A7%80%EC%82%AC%ED%95%AD/?slug=2025%ed%95%99%eb%85%84%eb%8f%84-2%ed%95%99%ea%b8%b0-sic-4%ec%bf%bc%ed%84%b0-%ec%a0%95%ea%b8%b0-%ea%b5%90%ed%99%98%ed%95%99%ec%83%9d-%ec%84%b8%eb%af%b8%eb%82%98-saf")
# # print(json.dumps(data, ensure_ascii=False, indent=4))

In [21]:
from datetime import datetime as dt
from datetime import date
import locale

def convert_korean_date_to_iso(korean_date_str):
    """
    'YYYY.MM.DD' 형식의 날짜 문자열을 ISO 8601 형식(YYYY-MM-DD)으로 변환합니다.
    """
    if not korean_date_str:
        return None

    # 형식: YYYY.MM.DD 또는 YYYY-MM-DD 등으로 들어올 수 있음.
    # 여기서는 '2025.11.04' 형식을 가정하고 파싱합니다.
    try:
        dt_object = dt.strptime(korean_date_str.strip().replace('/', '.').replace('-', '.'), '%Y.%m.%d')
        return dt_object.strftime('%Y-%m-%d')
    except ValueError:
        # 실패 시 None 반환
        return None

# --- 사용 예시 ---
# date_str_kor = '2025.11.04'
# iso_date = convert_korean_date_to_iso(date_str_kor)
# print(f"원본: {date_str_kor} -> ISO: {iso_date}") # 출력: 2025-11-04

In [23]:
from datetime import datetime as dt
from datetime import date

BASE_URL = "https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec%82%ac%ed%95%ad/"
TARGET_YEAR = 2025

def fetch_notices_all_pages(base_url, max_pages):
    """
    게시글 목록을 순회하며 2025년 이후의 항목만 필터링하여 메타데이터를 추출합니다.
    """
    all_notices_metadata = []

    # 날짜 필터링 기준 (2024년 12월 31일)
    cutoff_date = date(TARGET_YEAR - 1, 12, 31)

    for page_num in range(1, max_pages + 1):
        # URL 경로 설정: '/page/N/' 형태
        url = f"{base_url}page/{page_num}/" if page_num > 1 else base_url

        try:
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # 💡 정확한 선택자: 개별 게시글 항목을 나타내는 div
            notice_items = soup.select('div.row.no-gutters.align-items-center')

            if not notice_items:
                print(f"페이지 {page_num} ({url})에서 항목을 찾을 수 없어 종료합니다.")
                break

            should_break = False

            for item in notice_items:
                # 1. 게시일 추출 및 필터링
                date_element = item.select_one('.notice_col1 .h2')
                raw_date = date_element.get_text(strip=True) if date_element else None

                if raw_date:
                    try:
                        # 'YYYY.MM.DD' 형식을 datetime 객체로 파싱
                        post_date = dt.strptime(raw_date, '%Y.%m.%d').date()

                        # 2024년 이전 게시글이라면 순회 중단 플래그 설정
                        if post_date <= cutoff_date:
                            print(f"2024년 이전 게시글 발견 ({post_date}). 목록 순회 중단.")
                            should_break = True
                            break # item 순회 중단

                    except ValueError:
                        pass # 날짜 파싱 오류는 무시

                # 2. 2025년 이후 항목에 대해서만 메타데이터 추출
                try:
                    status_element = item.select_one('.notice_col2 .tag')
                    status = status_element.text.strip() if status_element else 'N/A'

                    link_element = item.select_one('.notice_col3 a')
                    link_href = link_element['href'].strip() if link_element else 'N/A'

                    category_element = link_element.select_one('.label')
                    category = category_element.text.strip() if category_element else 'N/A'

                    title_span = link_element.select_one('span[class*="d-inline-b"].m-pt-5')
                    title = title_span.text.strip() if title_span else '제목 없음'

                    all_notices_metadata.append({
                        '작성일': raw_date,
                        '진행상황': status,
                        '카테고리': category,
                        '제목': title,
                        '링크': link_href,
                    })

                except Exception:
                    continue # 항목 파싱 오류 무시

            if should_break:
                break # 전체 페이지 순회 중단

            print(f"페이지 {page_num} 크롤링 완료. 다음 페이지로 진행합니다.")

        except requests.exceptions.RequestException as e:
            print(f"페이지 {page_num} ({url})를 가져오는 중 오류가 발생했습니다: {e}")
            break

    return all_notices_metadata


def crawl_and_save_rag_data(notices_metadata, filename="ssu_rag_data_2025.jsonl"):
    """
    수집된 목록 메타데이터를 순회하며 상세 크롤링을 수행하고 JSON Lines 파일로 저장합니다.
    """
    print(f"\n--- RAG 상세 크롤링 및 JSON Lines 파일 생성 시작 (총 {len(notices_metadata)}개) ---")

    with open(filename, 'w', encoding='utf-8') as f:
        for i, notice in enumerate(notices_metadata):
            link = notice['링크']

            # 1. 상세 크롤링 (본문 및 이미지 URL 획득)
            detailed_data = crawl_post_content(link)

            if not detailed_data:
                continue

            # 2. RAG 레코드 통합 및 포맷팅
            rag_record = {
                'source_url': link,
                'post_title': notice['제목'],
                'category': notice['카테고리'],
                'status': notice['진행상황'],
                'posted_date': convert_korean_date_to_iso(notice['작성일']), # ISO 변환
                'full_body_text': detailed_data['full_body_text'],
                'image_urls': detailed_data['image_urls']
            }

            # 3. JSON Lines 파일에 저장
            f.write(json.dumps(rag_record, ensure_ascii=False) + '\n')

            if (i + 1) % 100 == 0:
                 print(f"✅ {i+1}번째 항목까지 JSON Lines 저장 완료.")

    print(f"\n🎉 RAG 데이터가 '{filename}' 파일에 성공적으로 저장되었습니다.")


if __name__ == '__main__':
    MAX_PAGES = 80

    print(f"--- 숭실대학교 공지사항 크롤링 시작 ({TARGET_YEAR}년 이후 필터링 및 페이지네이션) ---")

    metadata_results = fetch_notices_all_pages(BASE_URL, MAX_PAGES)

    if metadata_results:
        print(f"\n✅ 최종적으로 총 {len(metadata_results)}개의 ({TARGET_YEAR}년 이후) 공지사항 메타데이터를 수집했습니다.")

        # 2단계: 상세 크롤링 및 JSON Lines 저장
        crawl_and_save_rag_data(metadata_results)

    else:
        print("공지사항을 찾지 못했거나 크롤링에 실패했습니다. URL과 CSS Selector를 다시 확인해 주세요.")





--- 숭실대학교 공지사항 크롤링 시작 (2025년 이후 필터링 및 페이지네이션) ---
페이지 1 크롤링 완료. 다음 페이지로 진행합니다.
페이지 2 크롤링 완료. 다음 페이지로 진행합니다.
페이지 3 크롤링 완료. 다음 페이지로 진행합니다.
페이지 4 크롤링 완료. 다음 페이지로 진행합니다.
페이지 5 크롤링 완료. 다음 페이지로 진행합니다.
페이지 6 크롤링 완료. 다음 페이지로 진행합니다.
페이지 7 크롤링 완료. 다음 페이지로 진행합니다.
페이지 8 크롤링 완료. 다음 페이지로 진행합니다.
페이지 9 크롤링 완료. 다음 페이지로 진행합니다.
페이지 10 크롤링 완료. 다음 페이지로 진행합니다.
페이지 11 크롤링 완료. 다음 페이지로 진행합니다.
페이지 12 크롤링 완료. 다음 페이지로 진행합니다.
페이지 13 크롤링 완료. 다음 페이지로 진행합니다.
페이지 14 크롤링 완료. 다음 페이지로 진행합니다.
페이지 15 크롤링 완료. 다음 페이지로 진행합니다.
페이지 16 크롤링 완료. 다음 페이지로 진행합니다.
페이지 17 크롤링 완료. 다음 페이지로 진행합니다.
페이지 18 크롤링 완료. 다음 페이지로 진행합니다.
페이지 19 크롤링 완료. 다음 페이지로 진행합니다.
페이지 20 크롤링 완료. 다음 페이지로 진행합니다.
페이지 21 크롤링 완료. 다음 페이지로 진행합니다.
페이지 22 크롤링 완료. 다음 페이지로 진행합니다.
페이지 23 크롤링 완료. 다음 페이지로 진행합니다.
페이지 24 크롤링 완료. 다음 페이지로 진행합니다.
페이지 25 크롤링 완료. 다음 페이지로 진행합니다.
페이지 26 크롤링 완료. 다음 페이지로 진행합니다.
페이지 27 크롤링 완료. 다음 페이지로 진행합니다.
페이지 28 크롤링 완료. 다음 페이지로 진행합니다.
페이지 29 크롤링 완료. 다음 페이지로 진행합니다.
페이지 30 크롤링 완료. 다음 페이지로 진행합니다.
페이지 31 크롤링 완료. 다음 페이지로 진행합니다.
페이지 32 크롤링 완료. 다음 페이지로 진행합니다.

In [24]:
import json
import os

def print_jsonl_details(file_path, num_records=5):
    """JSON Lines 파일에서 처음 N개의 레코드를 읽어 세부 내용을 출력합니다."""

    if not os.path.exists(file_path):
        print(f"오류: 파일 '{file_path}'를 찾을 수 없습니다. 경로를 확인해 주세요.")
        return

    print(f"\n--- 📄 '{file_path}' 파일의 처음 {num_records}개 레코드 출력 ---")

    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= num_records:
                break

            try:
                # 1. 각 줄을 JSON 객체로 파싱
                doc = json.loads(line)

                # 2. 필수 필드를 포함하여 출력 형식 지정
                print(f"\n[레코드 #{i+1}]")
                print(f"  제목: {doc.get('post_title', 'N/A')}")
                print(f"  URL: {doc.get('source_url', 'N/A')}")
                print(f"  작성일: {doc.get('posted_date', 'N/A')}")
                print(f"  상태: {doc.get('status', 'N/A')}")
                print(f"  이미지 수: {len(doc.get('image_urls', []))}개")

                # 3. 본문 텍스트는 너무 길 수 있으므로 일부만 출력
                body_text = doc.get('full_body_text', '본문 없음')
                print(f"  본문: {body_text}")

            except json.JSONDecodeError:
                print(f"경고: {i+1}번째 줄은 유효한 JSON 형식이 아닙니다. 건너뜁니다.")

# --- 실행 ---
FILE_NAME = 'ssu_rag_data_2025.jsonl'
print_jsonl_details(FILE_NAME, num_records=5)


--- 📄 'ssu_rag_data_2025.jsonl' 파일의 처음 5개 레코드 출력 ---

[레코드 #1]
  제목: 2026년도 (재)전기공사공제조합장학회 제21기 장학생 선발 안내
  URL: https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec%82%ac%ed%95%ad/?f&category&paged=1&slug=2026%EB%85%84%EB%8F%84-%EC%9E%AC%EC%A0%84%EA%B8%B0%EA%B3%B5%EC%82%AC%EA%B3%B5%EC%A0%9C%EC%A1%B0%ED%95%A9%EC%9E%A5%ED%95%99%ED%9A%8C-%EC%A0%9C21%EA%B8%B0-%EC%9E%A5%ED%95%99%EC%83%9D-%EC%84%A0&keyword
  작성일: 2025-11-04
  상태: 진행
  이미지 수: 1개
  본문: 

[레코드 #2]
  제목: [중앙도서관] ‘천선란’ 작가 저자강연회 참가자 모집(~11/21)
  URL: https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec%82%ac%ed%95%ad/?f&category&paged=1&slug=%EC%A4%91%EC%95%99%EB%8F%84%EC%84%9C%EA%B4%80-%EC%B2%9C%EC%84%A0%EB%9E%80-%EC%9E%91%EA%B0%80-%EC%A0%80%EC%9E%90%EA%B0%95%EC%97%B0%ED%9A%8C-%EC%B0%B8%EA%B0%80%EC%9E%90-%EB%AA%A8&keyword
  작성일: 2025-11-04
  상태: 진행
  이미지 수: 2개
  본문: 슈패스 **신청기간**: 11/3(월) ~ 21(금) ‘저자강연회’ 검색 후 신청*선착순마감
안녕하세요. 중앙도서관입니다.
2025학년도 2학기 저자강연회에는
SF소설을 통해 따뜻한 상상력을 전하는천선란작가님을 강연자로 모셨습니다.
작가님이 들려주는 글과 SF의 세계로 여러분을 초대합니다.
[ 저자강연회 참가 안내